In [ ]:
import datajoint as dj

In [ ]:
import requests

In [ ]:
dj.conn()

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
schema = dj.schema('states')

In [ ]:
states = dict(
    AL='Alabama',        AK='Alaska',       AZ='Arizona',      AR='Arkansas',
    CA='California',     CO='Colorado',     CT='Connecticut',  DE='Delaware',
    FL='Florida',        GA='Georgia',      HI='Hawaii',       ID='Idaho', 
    IL='Illinois',       IN='Indiana',      IA='Iowa',         KS='Kansas',
    KY='Kentucky',       LA='Louisiana',    ME='Maine',        MD='Maryland',
    MA='Massachusetts',  MI='Michigan',     MN='Minnesota',    MS='Mississippi',
    MO='Missouri',       MT='Montana',      NE='Nebraska',     NV='Nevada',
    NH='New Hampshire',  NJ='New Jersey',   NM='New Mexico',   NY='New York',
    NC='North Carolina', ND='North Dakota', OH='Ohio',         OK='Oklahoma',
    OR='Oregon',         PA='Pennsylvania', RI='Rhode Island', SC='South Carolina',
    SD='South Dakota',   TN='Tennessee',    TX='Texas',        UT='Utah',
    VT='Vermont',        VA='Virginia',     WA='Washington',   WV='West Virginia', 
    WI='Wisconsin',      WY='Wyoming')

In [ ]:
@schema
class State(dj.Lookup):
    definition = """
    # United States
    state_code : char(2)
    ---
    state : varchar(20)
    """
    contents = states.items()

In [ ]:
State()

In [ ]:
def download(url):
    path = url.split('/') [-1]
    with open(path, 'wb') as f:
        f.write(requests.get(url).content)
        return path

In [ ]:
download("http://www.theus50.com/images/state-birds/{state}-bird.jpg")

In [ ]:
@schema
class StateBird(dj.Imported):
    definition = """
    -> State
    ---
    bird_image : longblob    
    """
    
    url_template = "http://www.theus50.com/images/state-birds/{state}-bird.jpg"
    
    def make(self, key):
        # fetch data upstream
        state = (State & key).fetch1('state')
        
        # compute
        url = self.url_template.format(state=state.lower().replace(' ', ''))
        temp_file = 'tmp.jpg'
        with open(temp_file, 'wb') as f:
            f.write(requests.get(url).content)
        
        # insert into self
        self.insert1(dict(key, bird_image=plt.imread(temp_file)))

In [ ]:
StateBird().populate(display_progress=True, suppress_errors=True)

In [ ]:
StateBird()

In [ ]:
img = (StateBird & {'state_code': "TX"}).fetch1('bird_image')

In [ ]:
plt.imshow(img)

In [ ]:
download("http://www.theus50.com/images/state-flags/{state}-flag.jpg")

In [ ]:
@schema
class StateFlag(dj.Imported):
    definition = """
    -> State
    ---
    flag_image : longblob    
    """
    
    url_template = "http://www.theus50.com/images/state-flags/{state}-flag.jpg"
    
    def make(self, key):
        state = (State & key).fetch1('state')
        url = self.url_template.format(state=state.lower().replace(' ', ''))
        temp_file = 'tmp.jpg'
        with open(temp_file, 'wb') as f:
            f.write(requests.get(url).content)
        self.insert1(dict(key, flag_image=plt.imread(temp_file)))

In [ ]:
img = (StateFlag & {'state_code': "TX"}).fetch1('flag_image')

In [ ]:
plt.imshow(img)

In [ ]:
StateFlag.populate(display_progress=True, suppress_errors=True)

In [ ]:
download("http://www.theus50.com/images/state-flowers/{state}-flower.jpg")

In [ ]:
@schema
class StateFlower(dj.Imported):
    definition = """
    -> State
    ---
    flower_image : longblob    
    """
    
    url_template = "http://www.theus50.com/images/state-flowers/{state}-flower.jpg"
    
    def make(self, key):
        state = (State & key).fetch1('state')
        url = self.url_template.format(state=state.lower().replace(' ', ''))
        temp_file = 'tmp.jpg'
        with open(temp_file, 'wb') as f:
            f.write(requests.get(url).content)
        self.insert1(dict(key, flower_image=plt.imread(temp_file)))

In [ ]:
StateFlower.populate(display_progress=True)

In [ ]:
plt.imshow((StateFlag & {'state_code': 'WA'}).fetch1('flag_image'))
plt.axis(False);

In [ ]:
StateFlag * State

In [ ]:
fig, axx = plt.subplots(5, 10, figsize=(16, 7))

for ax, info in zip(axx.flatten(), 
                    (StateFlag*State).fetch(as_dict=True)):
    ax.imshow(info['flag_image'])
    ax.set_title(info['state'])
    ax.axis(False)

In [ ]:
fig, axx = plt.subplots(5, 10, figsize=(16, 7))
for ax, info in zip(axx.flatten(), 
                    (StateBird*State).fetch(as_dict=True)):
    ax.imshow(info['bird_image'])
    ax.set_title(info['state'])
    ax.axis(False)

In [ ]:
fig, axx = plt.subplots(5, 10, figsize=(16, 7))
for ax, info in zip(axx.flatten(), 
                    (StateFlower*State).fetch(as_dict=True)):
    ax.imshow(info['flower_image'])
    ax.set_title(info['state'])
    ax.axis(False)

In [ ]:
@schema
class FlagSaturation(dj.Computed):
    definition = """
    -> StateFlag
    ---
    saturation :  float 
    """
    
    def make(self, key):
        img = (StateFlag & key).fetch1('flag_image')
        avg_color = img.mean(axis=(0,1))
        sat = avg_color.max()/avg_color.sum()
        self.insert1(dict(key, saturation=sat))
        

In [ ]:
FlagSaturation.populate()

In [ ]:
FlagSaturation * State * StateFlag

In [ ]:
fig, axx = plt.subplots(5, 10, figsize=(16, 7))

for ax, info in zip(axx.flatten(), 
                    (StateFlag*State*FlagSaturation).fetch(as_dict=True, order_by='saturation')):
    ax.imshow(info['flag_image'])
    ax.set_title(info['state'])
    ax.axis(False)

In [ ]:
@schema
class FlagSaturation(dj.Computed):
    definition = """
    -> StateFlag
    ---
    saturation :  float 
    """
    
    def make(self, key):
        img = (StateFlag & key).fetch1('flag_image')
        avg_color = img.mean(axis=(0,1))
        sat = avg_color.max()/avg_color.sum()
        self.insert1(dict(key, saturation=sat))
        

In [ ]:
StateFlower.populate()
StateFlag.populate()
StateBird.populate()
FlagSaturation.populate()

In [ ]:
@schema
class FlagBrightness(dj.Computed):
    definition = """
    -> StateFlag
    ---
    brightness :  float 
    """
    
    def make(self, key):
        img = (StateFlag & key).fetch1('flag_image')
        self.insert1(dict(key, brightness=img.mean()))

In [ ]:
FlagBrightness.populate()

In [ ]:
fig, axx = plt.subplots(5, 10, figsize=(16, 7))

for ax, info in zip(axx.flatten(), 
                    (StateFlag*State*FlagBrightness).fetch(as_dict=True, order_by='brightness')):
    ax.imshow(info['flag_image'])
    ax.set_title(info['state'])
    ax.axis(False)

In [ ]:
@schema
class FlagContrast(dj.Computed):
    definition = """
    -> FlagBrightness
    ---
    contrast :  float 
    """
    
    def make(self, key):
        img, brightness = (StateFlag * FlagBrightness & key).fetch1('flag_image', 'brightness')
        self.insert1(dict(key, contrast=img.mean(axis=-1).std()/brightness))

In [ ]:
FlagContrast.populate(display_progress=True)

In [ ]:
fig, axx = plt.subplots(5, 10, figsize=(16, 7))

for ax, info in zip(axx.flatten(), 
                    (StateFlag*State*FlagContrast).fetch(as_dict=True, order_by='contrast')):
    ax.imshow(info['flag_image'])
    ax.set_title(info['state'])
    ax.axis(False)

In [ ]:
@schema
class FlagHue(dj.Computed):
    definition = """
    -> FlagSaturation
    ---
    hue :  float 
    """
    
    def make(self, key):
        img, saturation = (StateFlag * FlagSaturation & key).fetch1('flag_image', 'saturation')
        self.insert1(dict(key, hue=img.mean(axis = -1).std()/saturation))
        

In [ ]:
FlagHue.populate(display_progress=True)

In [ ]:
fig, axx = plt.subplots(5, 10, figsize=(16, 7))

for ax, info in zip(axx.flatten(), 
                    (StateFlag*State*FlagHue).fetch(as_dict=True, order_by='hue')):
    ax.imshow(info['flag_image'])
    ax.set_title(info['state'])
    ax.axis(False)

In [ ]:
dj.Diagram(schema)